In [75]:
import numpy as np
import pandas as pd

import nltk

In [211]:
# df_train = pd.read_csv("DrugReviewAnalysis/drugsComTrain_raw.tsv", sep="\t")
# df_test = pd.read_csv("DrugReviewAnalysis/drugsComTest_raw.tsv", sep="\t")

df_train = pd.read_csv("drugsComTrain_raw.tsv", sep="\t")
df_test = pd.read_csv("drugsComTest_raw.tsv", sep="\t")

In [212]:
df_train = df_train[df_train["condition"].str.contains("users found this comment helpful.")==False]
df_test = df_test[df_test["condition"].str.contains("users found this comment helpful.")==False]

In [213]:
len(set(df_train["condition"]) & set(df_test["condition"])), len(set(df_test["condition"]))

(639, 664)

In [214]:
len(set(df_train["drugName"]) & set(df_test["drugName"])), len(set(df_test["drugName"]))

(2387, 2629)

In [168]:
df_train["timestamp"] = df_train["date"].apply(lambda x: pd.Timestamp(x))
df_test["timestamp"] = df_test["date"].apply(lambda x: pd.Timestamp(x))

In [169]:
df_train = df_train.drop(columns="Unnamed: 0")

In [170]:
df_test = df_test.drop(columns="Unnamed: 0")

In [171]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [172]:
# pip install autocorrect

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [173]:
from autocorrect import Speller

In [174]:
spell = Speller()

In [175]:
def preprocess(text):
    tokens = word_tokenize(text.lower())

    filtered_tokens = [token for token in tokens if token not in stopwords.words("english")]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    processed_text = " ".join(lemmatized_tokens)
    return processed_text


In [85]:
#df_train["condition"] = df_train["condition"].head(5).apply(lambda x: spell(x))

In [176]:
df_train["condition"] = df_train["condition"].apply(lambda x: str(x).lower())

In [177]:
np.sort(df_train["condition"].unique())

array(['abdominal distension', 'abnormal uterine bleeding', 'abortion',
       'acetaminophen overdose', 'acial lipoatrophy', 'acial wrinkles',
       'acne', 'actinic keratosis', 'actor ix deficiency',
       'acute coronary syndrome', 'acute lymphoblastic leukemia',
       'acute nonlymphocytic leukemia', 'acute promyelocytic leukemia',
       "addison's disease", 'adhd', 'adrenocortical insufficiency',
       'adult human growth hormone deficiency', 'aggressive behavi',
       'agitated state', 'agitation', 'aids related wasting',
       'ailure to thrive', 'alcohol dependence', 'alcohol withdrawal',
       'allergic reactions', 'allergic rhinitis', 'allergic urticaria',
       'allergies', 'alopecia', 'alpha-1 proteinase inhibitor deficiency',
       "alzheimer's disease", 'amebiasis', 'amenorrhea',
       'amilial cold autoinflammatory syndrome',
       'amilial mediterranean feve', 'amyotrophic lateral sclerosis',
       'anal fissure and fistula', 'anal itching', 'anaphylaxis',


In [178]:
# df_train.to_csv("DrugReviewAnalysis/drugsTrain_processed.csv", sep="\t", index=False)

df_train.to_csv("drugsTrain_processed.csv", sep="\t", index=False)

**So far only ran for the `condition` column of the training dataset. Took more than an hour to spellcheck the column with the current code, but using the nltk library runs much faster. Need to figure out how to speed up spell checking to use on the `review` feature of the dataset.**

In [180]:
df_train[df_train["condition"].str.contains("addison")]

,drugName,condition,review,rating,date,usefulCount,timestamp
1195,Decadron,addison's disease,"""Longer acting steroid option.\r\nGained weigh...",8.0,"August 6, 2017",0,2017-08-06
21819,Dexamethasone,addison's disease,"""Smooth feel and effective. Start slow 0.5 mg ...",9.0,"November 13, 2012",9,2012-11-13
23097,Dexamethasone,addison's disease,"""Taking this medication 0.25 mg daily with 0.0...",10.0,"April 5, 2013",11,2013-04-05
25357,Hydrocortisone,addison's disease,"""This medication is life sustaining. I take a...",10.0,"October 3, 2017",0,2017-10-03
48790,Hydrocortisone,addison's disease,"""Added so much weight,due to this drugs""",8.0,"January 10, 2016",4,2016-01-10
67629,Dexamethasone,addison's disease,"""I had weight gain.""",9.0,"December 8, 2013",4,2013-12-08
84321,Hydrocortone,addison's disease,"""Added so much weight,due to this drugs""",8.0,"January 10, 2016",4,2016-01-10
110102,Hydrocortone,addison's disease,"""Feel better,but getting really fat""",8.0,"February 9, 2015",7,2015-02-09
118723,Hydrocortisone,addison's disease,"""Feel better,but getting really fat""",8.0,"February 9, 2015",7,2015-02-09
122902,Fludrocortisone,addison's disease,"""Works like a charm and I have zero side effec...",10.0,"July 29, 2009",37,2009-07-29


In [181]:
df_train["review"] = df_train["review"].str.replace("&#039;", "'")

In [182]:
np.sort(df_train["condition"].unique())

array(['abdominal distension', 'abnormal uterine bleeding', 'abortion',
       'acetaminophen overdose', 'acial lipoatrophy', 'acial wrinkles',
       'acne', 'actinic keratosis', 'actor ix deficiency',
       'acute coronary syndrome', 'acute lymphoblastic leukemia',
       'acute nonlymphocytic leukemia', 'acute promyelocytic leukemia',
       "addison's disease", 'adhd', 'adrenocortical insufficiency',
       'adult human growth hormone deficiency', 'aggressive behavi',
       'agitated state', 'agitation', 'aids related wasting',
       'ailure to thrive', 'alcohol dependence', 'alcohol withdrawal',
       'allergic reactions', 'allergic rhinitis', 'allergic urticaria',
       'allergies', 'alopecia', 'alpha-1 proteinase inhibitor deficiency',
       "alzheimer's disease", 'amebiasis', 'amenorrhea',
       'amilial cold autoinflammatory syndrome',
       'amilial mediterranean feve', 'amyotrophic lateral sclerosis',
       'anal fissure and fistula', 'anal itching', 'anaphylaxis',


In [186]:
df_train[df_train["condition"] == "mist ("]

,drugName,condition,review,rating,date,usefulCount,timestamp
107305,Formoterol,mist (,"""I have used this for about 2 years and it sur...",9.0,"August 20, 2011",23,2011-08-20


In [142]:
import re

In [148]:
# pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.5 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 31.4 MB 7.0 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [192]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

In [193]:
dn_train_ohe = ohe.fit_transform(df_train["drugName"].values.reshape(-1,1)).toarray()
cond_train_ohe = ohe.fit_transform(df_train["condition"].values.reshape(-1,1)).toarray()

In [194]:
dn_train_ohe.shape, cond_train_ohe.shape

((159498, 3412), (159498, 811))

In [195]:
dn_test_ohe = ohe.fit_transform(df_test["drugName"].values.reshape(-1,1)).toarray()
cond_test_ohe = ohe.fit_transform(df_test["condition"].values.reshape(-1,1)).toarray()

In [196]:
dn_test_ohe.shape, cond_test_ohe.shape

((53200, 2629), (53200, 664))

In [205]:
df_train["pair"] = list(df_train["drugName"] + " " + df_train["condition"])


In [206]:
df_train.head()

,drugName,condition,review,rating,date,usefulCount,timestamp,pair
0,Valsartan,left ventricular dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,2012-05-20,Valsartan left ventricular dysfunction
1,Guanfacine,adhd,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,2010-04-27,Guanfacine adhd
2,Lybrel,birth control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,2009-12-14,Lybrel birth control
3,Ortho Evra,birth control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,2015-11-03,Ortho Evra birth control
4,Buprenorphine / naloxone,opiate dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,2016-11-27,Buprenorphine / naloxone opiate dependence


In [209]:
pairs = ohe.fit_transform(df_train["pair"].values.reshape(-1,1)).toarray()
pairs.shape

(159498, 7819)